<a href="https://colab.research.google.com/github/pineapple-666/Detect-Sepsis-in-Patients-Before-it-Emerges/blob/main/notebook/Step1_Data_Exploring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_auc_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

### Importing and splitting the data into training and testing



In [2]:
url = "https://raw.github.com/pineapple-666/Detect-Sepsis-in-Patients-Before-it-Emerges/main/data/train.csv.zip"
raw_data = pd.read_csv(url)
print(raw_data.head(3))

   patient  time    HR  O2Sat   Temp    SBP   MAP   DBP  Resp  EtCO2  ...  \
0    16426     1   NaN    NaN    NaN    NaN   NaN   NaN   NaN    NaN  ...   
1    16426     2  78.0   99.5  35.25  108.5  84.5  69.0  12.0    NaN  ...   
2    16426     3  80.0   99.0  36.20  113.0  87.0  73.0  12.0    NaN  ...   

    WBC  Fibrinogen  Platelets    Age  Gender  Unit1  Unit2  HospAdmTime  \
0   NaN         NaN        NaN  57.03     1.0    0.0    1.0        -4.25   
1  14.2         NaN        NaN  57.03     1.0    0.0    1.0        -4.25   
2   NaN         NaN        NaN  57.03     1.0    0.0    1.0        -4.25   

   ICULOS  SepsisLabel  
0     1.0          0.0  
1     2.0          0.0  
2     3.0          0.0  

[3 rows x 43 columns]


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088197 entries, 0 to 1088196
Data columns (total 43 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   patient           1088197 non-null  int64  
 1   time              1088197 non-null  int64  
 2   HR                980967 non-null   float64
 3   O2Sat             947133 non-null   float64
 4   Temp              369212 non-null   float64
 5   SBP               931096 non-null   float64
 6   MAP               953029 non-null   float64
 7   DBP               749357 non-null   float64
 8   Resp              921122 non-null   float64
 9   EtCO2             41511 non-null    float64
 10  BaseExcess        58822 non-null    float64
 11  HCO3              45312 non-null    float64
 12  FiO2              90753 non-null    float64
 13  pH                75490 non-null    float64
 14  PaCO2             60609 non-null    float64
 15  SaO2              37520 non-null    float64
 16  

In [8]:
raw_data['patient'].unique().size

28235

In [13]:
import random
random.seed(1)
num_patients_20_percent = 28235 * 0.2
random_numbers = random.sample(range(28236), int(num_patients_20_percent))

In [16]:
train_df = raw_data[raw_data['patient'].isin(random_numbers)].copy()
valid_df = raw_data[~raw_data['patient'].isin(random_numbers)].copy()

print(train_df.shape)
print(valid_df.shape)

(217132, 43)
(871065, 43)


In [18]:
total_rows_split = len(train_df) + len(valid_df)
total_rows_raw = len(raw_data)
print(total_rows_split)
print(total_rows_raw)

1088197
1088197


In [21]:
print(train_df.shape[0] / raw_data.shape[0])

0.19953372413267084
